## In order to create a spotify go to https://developer.spotify.com/documentation/web-api/tutorials/getting-started

## 1. INSTALL DEPENDANCIES
  

In [ ]:
!pip install -r requirements.txt

## 2 Import packages and set up pandas options

In [ ]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to a new line
pd.set_option('display.max_colwidth', 150)

## 3 LOAD ENVIRONMENT VARIABLES INTO YOUR PYTHON CODE

    !important: do not forget to create a ".env" file as a copy of example.env and add your own 
    SPOTIFY SPOTIFY_CLIENT_ID;SPOTIFY_CLIENT_SECRET

In [ ]:
# load environment variables

load_dotenv(override=True)
SPOTIFY_OAUTH_ENDPOINT=os.getenv('SPOTIFY_OAUTH_ENDPOINT')
SPOTIFY_API_ENDPOINT = os.getenv('SPOTIFY_API_ENDPOINT')
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')



## 4- GET ACCESS TOKEN FROM SPOTIFY
in order to connect to spotify api you need to obtain an access token by calling this function

In [ ]:
def get_token(endpoint, clientId, clientSecret):
    '''
        returns spotify access_token
    '''
    grant_type = 'client_credentials'
    # parameters of the API call
    body_params = {
        'client_id': clientId,
        'client_secret': clientSecret,
        'grant_type' : grant_type
        }

    url=f'{endpoint}/api/token' # formating the full endpoint
    try:
        response = requests.post(url, data=body_params) # making request to spotify to get a token
        token_raw = json.loads(response.text)
        token = token_raw["access_token"] # extracting access_token from api result
        return token
    except Exception as ex :
        print(f"Got an authentification error {ex}")

In [ ]:

def find_tracks_by_artiste(artist):

    # Get access token
    access_token = get_token(SPOTIFY_OAUTH_ENDPOINT, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)

    # API CALL
    search_response = requests.get(
        url = f'{SPOTIFY_API_ENDPOINT}/v1/search',
        headers={
            'Authorization': f'Bearer {access_token}'
        },
        params={
            'q': f'artist:{artist}',
            'type': 'track',
            'market': 'MR',
        }
    )
    # Parse result
    tracks = search_response.json().get('tracks', {}).get('items', [])
    return tracks

def miliseconds_to_minutes(millis):
    '''
        a helper function to convert miliseconds to minutes and seconds
    '''
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)
    return  f"{minutes} m {seconds} s"


def get_most_popular_songs(tracks):
    '''
        Get top 10 popular songs from a tracks list
    '''

    songs = []
    for  track in tracks:
        artists = ', '.join(artist['name'] for artist in track.get('artists', []))
        spotify_url = track.get('external_urls', {}).get('spotify')
        full_name = f"{track.get('name')} by {artists}"
        
        extracted_data = {
            "popularity": track.get('popularity'),
            "spotify_url": str(spotify_url),
            "full_name": full_name,
            "release_date": track.get('album', {}).get('release_date'),
            "duration_m": miliseconds_to_minutes(track.get('duration_ms'))

            }
        songs.append(extracted_data)

    songs_df = pd.DataFrame(songs)
    songs_df = songs_df.sort_values(by='popularity', ascending=False)
    return songs_df.head(10)

In [ ]:
import matplotlib.pyplot as plt


def display_songs_chart(songs_df):
    '''
        Display a bar chart of the top 10 More popular songs
        
    '''
    fig, ax = plt.subplots()
    fig.autofmt_xdate(rotation=45)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    songs_df = songs_df.head(10)
    ax.bar(songs_df["full_name"], songs_df["popularity"], color=colors)

    ax.set_ylabel('popularity %')
    ax.set_title('Song Bame')
    ax.legend(title='Spotify Songs analysis')

    plt.show()


In [ ]:
def save_data_to_json_fil(filename, data):
    with open (filename, 'w') as f:
        f.write(json.dumps(data))

In [ ]:

artist = "Dj Dhaker"
songs = find_tracks_by_artiste(artist)
save_data_to_json_fil('dj_dhaker.json', songs)


# get most popular songs 

result = get_most_popular_songs(songs)
display_songs_chart(result)
print(result)

## Congrats you now know how to get data from an API and analyse it :-)